In [ ]:
%load_ext autoreload
%autoreload 2

# chat

> A simple interface for the Mistral API.

> This notebook follows the [fastai style guide](https://docs.fast.ai/dev/style.html). (perhaps move this to the index page only.)

In [ ]:
#| default_exp chat 

## API Key

Testing our API key to see check whether we have it working correctly.

In [ ]:
import os
API_KEY = os.environ.get('MISTRAL_API_KEY', 'KEY NOT FOUND')
message = 'Who is the most renowned French artist?'
h = {'Content-Type': 'application/json', 'Accept': 'application/json', 'Authorization': f'Bearer {API_KEY}'}
d = {'model': 'open-mistral-7b', 'messages': [{'role': 'user', 'content': message}]}

In [ ]:
import requests
URL = 'https://api.mistral.ai/v1/chat/completions'
r = requests.post(URL, headers=h, json=d); r.status_code

200

In [ ]:
r.json()['choices'][0]['message']['content']

"There are many renowned French artists across various artistic mediums, but some of the most famous include:\n\n1. Leonardo da Vinci (1452-1519): Although he was an Italian artist, he spent his last few years in France and had a significant impact on French art. He is known for his iconic paintings such as the Mona Lisa and The Last Supper.\n\n2. Claude Monet (1840-1926): He is considered the founder of French Impressionism, an art movement characterized by its emphasis on capturing the sensory impressions of a scene, especially as related to painting en plein air (outdoors). Some of his most famous works include Water Lilies and Impression, Sunrise.\n\n3. Pablo Picasso (1881-1973): Although Spanish, he spent a significant part of his life in France and is often associated with the country. He co-founded the Cubist movement, which revolutionized European painting and sculpture. Some of his most famous works include Les Demoiselles d'Avignon and Guernica.\n\n4. Edgar Degas (1834-1917):

Exploring how message history would work.

In [ ]:
m = 'How about from Britian?'
d['messages'][0]['content'] = m
r = requests.post(URL, headers=h, json=d)
r.json()['choices'][0]['message']['content']

It seems that prior interactions are not stored. Therefore, we will have to do this ourselves.

## Mistral Library

The MistralAI library allows three different ways to use the API.
- No streaming.
- With streaming.
- Async.

In [ ]:
#| export
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
client = MistralClient(api_key=API_KEY)
messages = [ChatMessage(role='user', content='What is the best French cheese?')]

### No Streaming

In [ ]:
m = 'open-mistral-7b'
chat_response = client.chat(model=m, messages=messages); chat_response

ChatCompletionResponse(id='04190adbeefd4270a90e2c0a304af703', object='chat.completion', created=1712844872, model='open-mistral-7b', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='Choosing the "best" French cheese is subjective as it depends on personal preferences. Here are a few popular and highly regarded French cheeses:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese from the Massif Central region. It\'s known for its strong, pungent flavor and distinctive veining.\n2. Comté: A nutty and slightly sweet cow\'s milk cheese from the Franche-Comté region. It\'s made using traditional methods and has a rich, buttery texture.\n3. Camembert: A soft, creamy cow\'s milk cheese from Normandy. It\'s famous for its earthy, mushroomy flavor and white mold rind.\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese from the Ile-de-France region. It\'s known for its velvety texture, mild, buttery flavor, and white mold rind.\n5. Munster: A pungent

In [ ]:
chat_response.choices[0].message.content

'Choosing the "best" French cheese is subjective as it depends on personal preferences. Here are a few popular and highly regarded French cheeses:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese from the Massif Central region. It\'s known for its strong, pungent flavor and distinctive veining.\n2. Comté: A nutty and slightly sweet cow\'s milk cheese from the Franche-Comté region. It\'s made using traditional methods and has a rich, buttery texture.\n3. Camembert: A soft, creamy cow\'s milk cheese from Normandy. It\'s famous for its earthy, mushroomy flavor and white mold rind.\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese from the Ile-de-France region. It\'s known for its velvety texture, mild, buttery flavor, and white mold rind.\n5. Munster: A pungent, smelly cow\'s milk cheese from the Alsace region. It\'s made using raw milk and has a runny, gooey texture and strong, sulfurous flavor.\n\nIf you\'re looking for a versatile cheese that can be used in cooking, I\'d recommen

No streaming returns the entire message at once.

Let's see whether history is handled through the provided API.

In [ ]:
messages.append(ChatMessage(role='user', content='What about the British variety?'))
chat_response = client.chat(model=m, messages=messages); chat_response.choices[0].message.content

'Determining the "best" French or British cheese is subjective and depends on personal preferences. Here are some popular and highly regarded options from both countries:\n\nFrench Cheeses:\n1. Roquefort: A blue-veined cheese made from sheep\'s milk. It has a strong, pungent aroma and a creamy texture.\n2. Comté: A nutty and slightly sweet cheese made from cow\'s milk. It has a firm texture and can be aged for different lengths of time.\n3. Camembert: A soft, white mold-ripened cheese made from cow\'s milk. It has a rich, earthy flavor and a creamy texture.\n4. Brie de Meaux: A soft, creamy cheese with a white rind and a mild, buttery flavor. It is often eaten with bread or used as a spread.\n5. Munster: A soft, smelly cheese with a pungent aroma and a runny, creamy texture. It is made from cow\'s milk and has a spicy, slightly sweet flavor.\n\nBritish Cheeses:\n1. Cheddar: A hard, sharp-tasting cheese made from cow\'s milk. It has a crumbly texture and can be aged for different length

It appearas not.

In [ ]:
?ChatMessage

Init signature:
ChatMessage(
    *,
    role: str,
    content: Union[str, List[str]],
    name: Optional[str] = None,
    tool_calls: Optional[List[mistralai.models.chat_completion.ToolCall]] = None,
) -> None
Docstring:     
Usage docs: https://docs.pydantic.dev/2.6/concepts/models/

A base class for creating Pydantic models.

Attributes:
    __class_vars__: The names of classvars defined on the model.
    __private_attributes__: Metadata about the private attributes of the model.
    __signature__: The signature for instantiating the model.

    __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.
    __pydantic_core_schema__: The pydantic-core schema used to build the SchemaValidator and SchemaSerializer.
    __pydantic_custom_init__: Whether the model has a custom `__init__` function.
    __pydantic_decorators__: Metadata containing the decorators defined on the model.
        This replaces `Model.__validators__` and `Model.__root_val

In [ ]:
?client.chat

Signature:
client.chat(
    messages: List[Any],
    model: Optional[str] = None,
    tools: Optional[List[Dict[str, Any]]] = None,
    temperature: Optional[float] = None,
    max_tokens: Optional[int] = None,
    top_p: Optional[float] = None,
    random_seed: Optional[int] = None,
    safe_mode: bool = False,
    safe_prompt: bool = False,
    tool_choice: Union[str, mistralai.models.chat_completion.ToolChoice, NoneType] = None,
    response_format: Union[Dict[str, str], mistralai.models.chat_completion.ResponseFormat, NoneType] = None,
) -> mistralai.models.chat_completion.ChatCompletionResponse
Docstring:
A chat endpoint that returns a single response.

Args:
    model (str): model the name of the model to chat with, e.g. mistral-tiny
    messages (List[Any]): messages an array of messages to chat with, e.g.
        [{role: 'user', content: 'What is the best French cheese?'}]
    tools (Optional[List[Function]], optional): a list of tools to use.
    temperature (Optional[float], 

In [ ]:
messages

[ChatMessage(role='user', content='What is the best French cheese?', name=None, tool_calls=None),
 ChatMessage(role='user', content='What about the British variety?', name=None, tool_calls=None)]

To implement history, we will need to store prior interactions. Each time a new message is sent in the conversation, all prior queries and responses will have to be sent.

In [ ]:
def get_response(message): return client.chat(model=m, messages=message).choices[0].message.content

In [ ]:
history = []
contents = 'What is the best French cheese?'
messages = history + [ChatMessage(role='user', content=contents)]
response = get_response(messages); response

'Determining the "best" French cheese is subjective as it depends on personal taste preferences. Here are some popular and highly regarded French cheeses across different categories:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese, known for its distinctive strong flavor and tangy taste.\n2. Camembert: A soft, creamy cow\'s milk cheese with a pungent aroma and earthy, mushroomy flavors.\n3. Comté: A nutty, sweet, and slightly fruity cow\'s milk cheese, often compared to Swiss Emmental.\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese with a rich, buttery, and mildly pungent flavor.\n5. Munster: A pungent, smelly cow\'s milk cheese with a gooey, runny texture when ripe.\n6. Reblochon: A soft, bloomy rind cheese with a rich, creamy texture and a mild but distinctive flavor.\n7. Cheddar de Jura: A hard, nutty cow\'s milk cheese with a complex, savory flavor, similar to English cheddar.\n8. Époisses: A pungent, soft, cow\'s milk cheese with a runny, gooey texture and a strong, sulf

In [ ]:
history.append(ChatMessage(role='user', content=contents)); history

[ChatMessage(role='user', content='What is the best French cheese?', name=None, tool_calls=None)]

In [ ]:
history.append(ChatMessage(role='assistant', content=response)); history

[ChatMessage(role='user', content='What is the best French cheese?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Determining the "best" French cheese is subjective as it depends on personal taste preferences. Here are some popular and highly regarded French cheeses across different categories:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese, known for its distinctive strong flavor and tangy taste.\n2. Camembert: A soft, creamy cow\'s milk cheese with a pungent aroma and earthy, mushroomy flavors.\n3. Comté: A nutty, sweet, and slightly fruity cow\'s milk cheese, often compared to Swiss Emmental.\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese with a rich, buttery, and mildly pungent flavor.\n5. Munster: A pungent, smelly cow\'s milk cheese with a gooey, runny texture when ripe.\n6. Reblochon: A soft, bloomy rind cheese with a rich, creamy texture and a mild but distinctive flavor.\n7. Cheddar de Jura: A hard, nutty cow\'s milk cheese with a complex, sa

In [ ]:
contents = 'What about the British variety?'
messages = history + [ChatMessage(role='user', content=contents)]; messages

[ChatMessage(role='user', content='What is the best French cheese?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Determining the "best" French cheese is subjective as it depends on personal taste preferences. Here are some popular and highly regarded French cheeses across different categories:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese, known for its distinctive strong flavor and tangy taste.\n2. Camembert: A soft, creamy cow\'s milk cheese with a pungent aroma and earthy, mushroomy flavors.\n3. Comté: A nutty, sweet, and slightly fruity cow\'s milk cheese, often compared to Swiss Emmental.\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese with a rich, buttery, and mildly pungent flavor.\n5. Munster: A pungent, smelly cow\'s milk cheese with a gooey, runny texture when ripe.\n6. Reblochon: A soft, bloomy rind cheese with a rich, creamy texture and a mild but distinctive flavor.\n7. Cheddar de Jura: A hard, nutty cow\'s milk cheese with a complex, sa

In [ ]:
response = get_response(messages); response

"British cheese offers a diverse range of flavors and textures. Here are some popular and highly regarded British cheeses:\n\n1. Cheddar: A hard, savory cow's milk cheese with a sharp, tangy flavor and a distinctive crumbly texture when aged.\n2. Stilton: A blue-veined cheese with a strong, pungent aroma and a creamy, crumbly texture. It is often described as having a distinctive, complex, and slightly salty taste.\n3. Wensleydale: A soft, creamy, and crumbly cheese with a mild and slightly sweet flavor, often paired with fruits or honey.\n4. Cheshire: A hard, crumbly cow's milk cheese with a tangy, slightly acidic flavor and a semi-sweet aftertaste.\n5. Red Leicester: A hard, orange-colored cow's milk cheese with a sharp, tangy flavor and a slightly sweet aftertaste.\n6. Lincolnshire Poacher: A hard, smoked cow's milk cheese with a rich, smoky flavor and a slightly sweet, buttery finish.\n7. Somerset Brie: A soft, creamy cow's milk cheese with a mild, buttery flavor and a bloomy rind.

In [ ]:
history.append(ChatMessage(role='user', content=contents))
history.append(ChatMessage(role='assistant', content=response))
history

[ChatMessage(role='user', content='What is the best French cheese?', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='Determining the "best" French cheese is subjective as it depends on personal taste preferences. Here are some popular and highly regarded French cheeses across different categories:\n\n1. Roquefort: A blue-veined sheep\'s milk cheese, known for its distinctive strong flavor and tangy taste.\n2. Camembert: A soft, creamy cow\'s milk cheese with a pungent aroma and earthy, mushroomy flavors.\n3. Comté: A nutty, sweet, and slightly fruity cow\'s milk cheese, often compared to Swiss Emmental.\n4. Brie de Meaux: A soft, creamy cow\'s milk cheese with a rich, buttery, and mildly pungent flavor.\n5. Munster: A pungent, smelly cow\'s milk cheese with a gooey, runny texture when ripe.\n6. Reblochon: A soft, bloomy rind cheese with a rich, creamy texture and a mild but distinctive flavor.\n7. Cheddar de Jura: A hard, nutty cow\'s milk cheese with a complex, sa

## Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()